In [10]:
import os
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
load_dotenv()

# SCOPES: Gmail read-only
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
token_path = os.getenv("token_path")
credentials_path = os.getenv("credentials_path")

In [11]:
def authenticate_gmail():
    creds = None
    flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('gmail', 'v1', credentials=creds)
    return service

#service = authenticate_gmail()

In [70]:
def get_message_metadata(service, user_id='me', max_results=5):
    results = service.users().messages().list(userId=user_id, maxResults=max_results, q="after:2025/08/08").execute()
    messages = results.get('messages', [])

    for _, msg in enumerate(messages):
        msg_data = service.users().messages().get(userId=user_id, id=msg['id'], format='metadata', metadataHeaders=['Subject', 'From', 'Date']).execute()
        headers = msg_data.get('payload', {}).get('headers', [])

        email_info = {header['name']: header['value'] for header in headers if header['name'] in ['Subject', 'From', 'Date']}
        print(f"From: {email_info.get('From')}")
        print(f"Subject: {email_info.get('Subject')}")
        print(f"Date: {email_info.get('Date')}")

In [72]:
# import json, collections

# out_dict=collections.defaultdict(list) #dict to store the relevant information
    
# for msg in l[0]:
#     start_time = time.time()
#     msg_data = service.users().messages().get(userId="me", id=msg["id"], format="full").execute()

#     payload = msg_data.get("payload", {})
#     # subject, sender = extract_headers(payload)
#     # body_text = decode_body(payload, prefer_plain=True).strip()

#     # print(f"\n--- Email ---")
#     # print(f"From: {sender}")
#     # print(f"Subject: {subject}")
#     # print(f"Body:\n{body_text}")
#     print(f"[Thread] >> Time taken: {time.time() - start_time:.4f} sec.")
#     out_dict["ID"].append(msg_data["id"])
#     out_dict["payload"].append(payload)


In [ ]:
import sys
out_dict={}
size_in_bytes=sys.getsizeof(out_dict)

size_in_mb=size_in_bytes / (1024 * 1024)
print(f"Size in MB: {size_in_mb}")

Size in MB: 0.00018310546875


In [74]:
import json, gzip

# Decompress and load
# with gzip.open('data.json.gz', 'rb') as f:
#     decompressed_bytes = f.read()
# decompressed_data = json.loads(decompressed_bytes.decode('utf-8'))
# print(decompressed_data)

In [8]:
from dags.utils import *
from datetime import  datetime, timedelta

In [12]:
def get_dates() -> list[tuple]:
    today = datetime.now().date()
    ranges = []
    for i in range(1, 10):
        after_date = today - timedelta(days=i)
        before_date = today - timedelta(days=i - 1)
        ranges.append((after_date.strftime("%Y/%m/%d"), before_date.strftime("%Y/%m/%d")))
    return ranges
dates =get_dates()
x = await async_get_ids_main(dates, token_path)

[CORO - 8] >> Time taken: 2.5456 sec.
[CORO - 9] >> Time taken: 2.5365 sec.
[CORO - 3] >> Time taken: 2.5675 sec.
[CORO - 6] >> Time taken: 2.5563 sec.
[CORO - 2] >> Time taken: 2.5708 sec.
[CORO - 5] >> Time taken: 2.5623 sec.
[CORO - 1] >> Time taken: 2.5755 sec.
[CORO - 7] >> Time taken: 2.5563 sec.
[CORO - 4] >> Time taken: 3.2818 sec.


In [13]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']    
creds = Credentials.from_authorized_user_file(token_path, SCOPES)
service = build('gmail', 'v1', credentials=creds)
msg = service.users().messages().list(userId='me', q=f"after:{dates[0][0]} before:{dates[0][1]}").execute()

In [17]:
ids = [dict_["id"]  for dict_ in msg.get('messages', [])]
print(ids)

['198aaf280d978c5c', '198a9884c0401181', '198a9328124ea8b7', '198a929e278667ed', '198a8ccdfab8794a', '198a8ca59f9772a2', '198a8c9b7f0cb680', '198a884249605c9a', '198a88147746e4d3', '198a8809279d1271', '198a8516f32ddf9d', '198a8122e42a966a', '198a7f1d8d11e51c', '198a7efe11efcd4b', '198a7ed8d08a7776', '198a7bf2528a891e', '198a7a40712f1b90', '198a76b760e90b33', '198a74e16ed2573a', '198a74a8634d6201', '198a73cb04e06123', '198a73621bbbfae0', '198a712047ddf7b0', '198a710359a7ded4', '198a6c7ef6a2a3f5', '198a6b260d1b6d50', '198a6a366ca11770', '198a68bed237dcc2', '198a6828a63d96a1']
